# Four-node NEM Capacity Expansion model

A simple, four node model of NEM to demonstrate the basic workflow of a capacity expansion modelling excercise, using PyPSA. 

This simplified model aims to introduce some of the basic concepts, and allow relatively quick exploration of the problem, by reducing the complexity.  In this case, we look at capacity expansion for (only) the year 2030, and consider only a handful of generators at each node (a mix of existing coal generators, and potential candidates for new capacity), and simplified interconnectors between the regions. 

The notebook is available on github here: https://github.com/dylanjmcconnell/four-node-nem

In [ ]:
import pypsa
from pypsa.common import annuity
import pandas as pd
import matplotlib.pyplot as plt
import helper

In [ ]:
#First, create network

network = pypsa.Network(name="four-node-nem")

#Add nodes
network.add("Bus", name="NSW", x=151.20, y=-33.87)
network.add("Bus", name="QLD", x=153.03, y=-27.47)
network.add("Bus", name="SA", x=138.63, y=-34.93)
network.add("Bus", name="VIC", x=145.01, y=-37.81)

In [ ]:
m = network.plot.map(boundaries = [138, 154, -45, -25], bus_size=.1)

In [ ]:
#Add carries (these are used for analysing the results, or adding custom constraints)

CARRIERS = {
    "solar": "#F8E71C", #"gold",
    "wind": "#417505", #"green",
    "gas": "#F48E1B", #"orange",
    "brown_coal": "#8B572A", #brown
    "black_coal": "black",
    "load": "slategrey",
    "AC": "violet",
    "bess": "blue"
}

network.add(
    "Carrier",
    CARRIERS.keys(),
    color=CARRIERS.values(),
)

## Adding snapshots and loads to the buses
 
- We then need to set the "snapshots". Snapshots are the main way time is represented in PyPSA.
- We then add load to each of the buses.

In this case, we by loading a data frame of load data, with a datetime index. We use the datetime index of this to create the snapshots. And then add the active power demand (`p_set`, in MW) at each time step from the dataframe to loads at each bus.


In [ ]:
#load the data
df_load = pd.read_csv("demand.csv", index_col=0, parse_dates=True)
display(df_load)

In [ ]:
# Set snapshots
network.set_snapshots(df_load.index)

# Add Load component
network.add("Load", 
            ["VIC", "SA", "NSW", "QLD"],
            suffix="_load", 
            bus=["VIC", "SA", "NSW", "QLD"],
            p_set=df_load[["VIC", "SA", "NSW", "QLD"]], 
            carrier="load")

In [ ]:
display(network)


## Adding existing generation to the buses. 

We then add generation to the buses. In this case, brown coal in Victoria, and black coal in NSW and QLD.  For existing plants, we specifiy the capacity (`p_nom`), as well as the marginal cost, ramp_rate_limits (and many other optional features).


In [ ]:
## Add brown coal 

network.add(
    "Generator",
    ["VIC"],
    suffix="_brown",
    bus=["VIC"],
    p_nom=3000,
    marginal_cost=20,
    carrier="brown_coal",
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    overwrite=True
)

## Add black coal 

network.add(
    "Generator",
    ["NSW", "QLD"],
    suffix="_black",
    bus=["NSW", "QLD"],
    p_nom=4000,
    marginal_cost=60,
    carrier="black_coal",
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    overwrite=True
)


In [ ]:
display(network.generators.T)

## New Generation

**For candidate new generation, we set p_nom_extendable=True**  

This is necessary for capacity expansion - rather than setting the p_nom capacity. This is the thing we are trying to determine, or optimise through this process. We set a capital cost (as well as marginal cost) - which is represented as an ammortised cost over the life of the project. 

For variable renewable resources, we pass the capacity factors as `p_max_pu` (per-unit (p.u.) time series that defines the maximum available output of a generator relative to its nominal capacity (`p_nom`) at each snapshot)



In [ ]:
#Load wind data
df_wind = pd.read_csv("wind.csv", index_col=0, parse_dates=True)
df_wind.head()

In [ ]:
network.add(
    "Generator",
    ["VIC", "SA", "NSW", "QLD"],
    suffix="_wind",
    bus=["VIC", "SA", "NSW", "QLD"],
    p_max_pu=df_wind[["VIC", "SA", "NSW", "QLD"]],
    p_nom_extendable=True,
    capital_cost=annuity(0.05, 30) * 2_800_000,
    carrier="wind")


In [ ]:
#Load solar data
df_solar = pd.read_csv("solar.csv", index_col=0, parse_dates=True)

network.add(
    "Generator",
    ["VIC", "SA", "NSW", "QLD"],
    suffix="_solar",
    bus=["VIC", "SA", "NSW", "QLD"],
    p_max_pu=df_solar[["VIC", "SA", "NSW", "QLD"]],
    p_nom_extendable=True,
    capital_cost=annuity(0.05, 30) * 1_100_000,
    carrier="solar")

In [ ]:
## Add gas generation - in this case we include a marginal cost
network.add(
    "Generator",
    ["VIC", "SA", "NSW", "QLD"],
    suffix="_gas",
    bus=["VIC", "SA", "NSW", "QLD"],
    p_nom_extendable=True,
    marginal_cost=120,
    capital_cost=annuity(0.05, 30) * 2_000_000,
    carrier="gas"
)


## Adding links between the nodes

We then add "links" between the nodes, to allow for transmission of power between the buses. (There are different ways interconnectors can be represented in PyPSA - we use the simpler "Link" component in this example). 

## Running the optimisation

In [ ]:
network.add(
        "Link",
        f"VIC-SA",
        bus0="VIC",
        bus1="SA",
        p_nom=1000,
        carrier="AC",
        p_min_pu=-1,  # bidirectional
        overwrite=True
    )

network.add(
        "Link",
        f"VIC-NSW",
        bus0="VIC",
        bus1="NSW",
        p_nom=2500,
        carrier="AC",
        p_min_pu=-1,  # bidirectional
        overwrite=True
    )

network.add(
        "Link",
        f"NSW-QLD",
        bus0="NSW",
        bus1="QLD",
        p_nom=1500,
        carrier="AC",
        p_min_pu=-1,  # bidirectional
        overwrite=True
    )


## Run the optimisation

This step can take some time!


In [ ]:
network.optimize()



## Exploring the results

In [ ]:
network.statistics()


In [ ]:
network.statistics.energy_balance().div(1e6).round(2).sort_values()

In [ ]:
average_cost = (
    (network.statistics.capex().sum() + network.statistics.opex().sum())
    / network.loads_t.p_set.sum().sum()
)
display(f"Average cost: {average_cost:.2f} AUD/MWh")

In [ ]:
network.buses_t.marginal_price.mean()


In [ ]:
#Capacity 
network.statistics.optimal_capacity.iplot()

In [ ]:
#Capacity 
network.statistics.energy_balance.iplot("bar")

In [ ]:
#Dispatch
network.statistics.energy_balance.iplot()


In [ ]:
# Maps
import cartopy.crs as ccrs
crs = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": crs})

# Use the energy balance statistics to prepare the bus sizes and plot the network
bus_size = network.statistics.energy_balance(groupby=["bus", "carrier"]).droplevel(
    "component")

d = network.plot(ax=ax, bus_size=bus_size / 4e7, margin=0.75, bus_split_circle=True, boundaries = [135, 160, -45, -20])








# Storage

Now we will add storage to a (further simplified) model.  In this case, we will look at a single region (Victoria)

### Base Victoria model:

In [ ]:
victoria = helper.build_network(regions=["VIC"])
victoria.optimize()

In [ ]:
victoria.statistics.optimal_capacity.iplot()

In [ ]:
victoria.statistics.energy_balance.iplot()

## Add storage

In this case we add a `StorageUnit`, and similarly ensure `p_nom_extendable` is True.  We need to set efficiency, as well as hours of storage, marginal cost, and capital cost)


(There are multiple ways you can add storage in PyPSA - including one approach where the hours of storage is also optimised). 

In [ ]:
victoria_storage = helper.build_network()

In [ ]:
victoria_storage.add(
            "StorageUnit",
            "VIC_BESS",
            bus="VIC",
            carrier="bess",
            max_hours=4,
            capital_cost=1_100_00,
            efficiency_store=.95,
            efficiency_dispatch=.95,
            p_nom_extendable=True,
            marginal_cost = 1,
            standing_loss=.001)

In [ ]:
victoria_storage.optimize()

In [ ]:
victoria_storage.statistics.energy_balance.iplot()

In [ ]:
victoria_storage.statistics.energy_balance.iplot("bar")

In [ ]:
victoria.statistics.energy_balance.iplot("bar")